In [1]:
import sqlite3
import pandas as pd
import csv
from flask import Flask, render_template_string, render_template
from werkzeug.serving import run_simple

df= pd.read_csv("TestPad_PCB_XYRGB_V2.csv")


In [2]:
con = sqlite3.connect('Printed_circuit.db')
cursor = con.cursor()

create_table = """CREATE TABLE IF NOT EXISTS Printed_ckt
                                    (ID INTPRIMARY KEY,
                                    X INT,
                                    Y INT,
                                    R REAL,
                                    G REAL,
                                    B REAL,
                                    Grey INT
                                    );
                                    """
cursor.execute(create_table)
df.to_sql('Printed_circuit', con, if_exists='replace', index=False)

con.commit()
con.close()




In [3]:
app = Flask(__name__,template_folder='.' )

@app.route('/')
def home():
    data_html = df.head(10).to_html(index=False, classes='table table-striped')
    return render_template_string('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Printed Circuit Board Processed Image</title>
        <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
        <style>
            body { padding-top: 50px; }
            .container { max-width: 800px; }
            nav a { margin-right: 10px; }
        </style>
    </head>
    <body>
        <nav class="navbar navbar-expand-lg navbar-light bg-light fixed-top">
            <a class="navbar-brand" href="/">PCB Process Image Project</a>
            <div class="collapse navbar-collapse" id="navbarNav">
                <ul class="navbar-nav">
                    <li class="nav-item"><a class="nav-link" href="/">Home</a></li>
                    <li class="nav-item"><a class="nav-link" href="/about">About</a></li>
                    <li class="nav-item"><a class="nav-link" href="/data">Data</a></li>
                </ul>
            </div>
        </nav>
        <div class="container">
            <h1 class="mt-5">We Are Welcoming You To Our Printed Circuit Board Process Image Project</h1>
            <div>{{data|safe}}</div>
        </div>
    </body>
    </html>
    ''', data=data_html)

@app.route('/about')
def about():
    return render_template_string('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>About</title>
        <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
        <style>
            body { padding-top: 50px; }
            .container { max-width: 800px; }
        </style>
    </head>
    <body>
        <nav class="navbar navbar-expand-lg navbar-light bg-light fixed-top">
            <a class="navbar-brand" href="/">PCB Process Image Project</a>
            <div class="collapse navbar-collapse" id="navbarNav">
                <ul class="navbar-nav">
                    <li class="nav-item"><a class="nav-link" href="/">Home</a></li>
                    <li class="nav-item"><a class="nav-link" href="/about">About</a></li>
                    <li class="nav-item"><a class="nav-link" href="/data">Data</a></li>
                </ul>
            </div>
        </nav>
        <div class="container">
            <h1 class="mt-5">About Printed Circuit Board Process Image</h1>
            <p>Source: <a href="https://archive.ics.uci.edu/dataset/990/printed+circuit+board+processed+image">This Is The Source Of Our Data</a></p>
            <p>Type of variables:</p>
            <ul>
                <li>X: Integer : The dataset includes X representing pixel positions</li>
                <li>Y: Integer : The dataset includes Y representing pixel positions</li>
                <li>R: Continuous Variable : R values determining pixel color (minmax normalized from 0-255)</li>
                <li>G: Continuous variable : G values determining pixel color (minmax normalized from 0-255)</li>
                <li>B: Continuous variable : B values determining pixel color (minmax normalized from 0-255)</li>
                <li>Grey: Binary : A 'Grey' field indicates approximate grey pixels</li>
            </ul>
        </div>
    </body>
    </html>
    ''')

@app.route('/data')
def data():
    con = sqlite3.connect('Printed_ckt.db')
    c = con.cursor()
    c.execute('SELECT * FROM Printed_ckt LIMIT 10')  
    data = c.fetchall()
    con.close()
    
    data_html = '<table class="table table-striped"><thead><tr><th>X</th><th>Y</th><th>R</th><th>G</th><th>B</th><th>Grey</th></tr></thead><tbody>'
    for row in data:
        data_html += '<tr>' + ''.join(f'<td>{col}</td>' for col in row) + '</tr>'
    data_html += '</tbody></table>'
    
    return render_template_string('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Data</title>
        <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
        <style>
            body { padding-top: 50px; }
            .container { max-width: 800px; }
        </style>
    </head>
    <body>
        <nav class="navbar navbar-expand-lg navbar-light bg-light fixed-top">
            <a class="navbar-brand" href="/">PCB Process Image Project</a>
            <div class="collapse navbar-collapse" id="navbarNav">
                <ul class="navbar-nav">
                    <li class="nav-item"><a class="nav-link" href="/">Home</a></li>
                    <li class="nav-item"><a class="nav-link" href="/about">About</a></li>
                    <li class="nav-item"><a class="nav-link" href="/data">Data</a></li>
                </ul>
            </div>
        </nav>
        <div class="container">
            <h1 class="mt-5">Data</h1>
            <div>{{data|safe}}</div>
        </div>
    </body>
    </html>
    ''', data=data_html)

In [ ]:
run_simple('localhost',9100,app,use_reloader=False, use_debugger=False)

 * Running on http://localhost:9100
Press CTRL+C to quit
127.0.0.1 - - [20/Jul/2024 23:24:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2024 23:24:30] "GET /about HTTP/1.1" 200 -
